In [7]:
#################### LOADING ########################

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
import cesium
from cesium import featurize


# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=RuntimeWarning)
simplefilter(action='ignore', category=FutureWarning)

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

%matplotlib inline

# variavel que evita NaN nos resultados
epslon = 0.00005
def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
   return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+ epslon))  

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

#função para normalização
def znorm(x):
  if np.std(x)!=0:
    x_znorm = (x - np.mean(x)) / np.std(x)
  else:
    x_znorm = (x - np.mean(x)) / (np.std(x) + epslon) 
   
  return x_znorm

#função para desnormatização
def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

def rolling_window_real(series, window):
  data = []
  for i in range(len(series)-window):
    example = (np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds



In [5]:
#### DEFs ####

def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados


#########################################################

features_to_use_all = [
    "amplitude",
    "anderson_darling",
    "flux_percentile_ratio_mid20",
    "flux_percentile_ratio_mid35",
    "flux_percentile_ratio_mid50",
    "flux_percentile_ratio_mid65",
    "flux_percentile_ratio_mid80",
    "percent_beyond_1_std",
    "percent_beyond_1_std",
    "period_fast",
    "qso_log_chi2_qsonu",
    "qso_log_chi2nuNULL_chi2nu",
    "shapiro_wilk",
    "stetson_j",
    "stetson_k",
    "weighted_average",
    "maximum",
    "max_slope",
    "median",
    "median_absolute_deviation",
    "percent_close_to_median",
    "minimum",
    "skew",
    "std",
    "weighted_average",
    "all_times_nhist_numpeaks",
    "all_times_nhist_peak1_bin",
    "all_times_nhist_peak2_bin",
    "all_times_nhist_peak3_bin",
    "all_times_nhist_peak4_bin",
    "all_times_nhist_peak_1_to_2",
    "all_times_nhist_peak_1_to_3",
    "all_times_nhist_peak_1_to_4",
    "all_times_nhist_peak_2_to_3",
    "all_times_nhist_peak_2_to_4",
    "all_times_nhist_peak_3_to_4",
    "all_times_nhist_peak_val",
    "avg_double_to_single_step",
    "avg_err",
    "avgt",
    "cad_probs_1",
    "cad_probs_10",
    "cad_probs_20",
    "cad_probs_30",
    "cad_probs_40",
    "cad_probs_50",
    "cad_probs_100",
    "cad_probs_500",
    "cads_avg",
    "cads_kurtosis",
    "cads_med",
    "cads_skew",
    "cads_std",
    "mean",
    "med_err",
    "n_epochs",
    "std_double_to_single_step",
    "std_err",
    "total_time",
    "fold2P_slope_10percentile",
    "fold2P_slope_90percentile",
    "freq1_amplitude1",
    "freq1_amplitude2",
    "freq1_amplitude3",
    "freq1_amplitude4",
    "freq1_freq",
    "freq1_lambda",
    "freq1_rel_phase2",
    "freq1_rel_phase3",
    "freq1_rel_phase4",
    "freq1_signif",
    "freq2_amplitude1",
    "freq2_amplitude2",
    "freq2_amplitude3",
    "freq2_amplitude4",
    "freq2_freq",
    "freq2_rel_phase2",
    "freq2_rel_phase3",
    "freq2_rel_phase4",
    "freq3_amplitude1",
    "freq3_amplitude2",
    "freq3_amplitude3",
    "freq3_amplitude4",
    "freq3_freq",
    "freq3_rel_phase2",
    "freq3_rel_phase3",
    "freq3_rel_phase4",
    "freq_amplitude_ratio_21",
    "freq_amplitude_ratio_31",
    "freq_frequency_ratio_21",
    "freq_frequency_ratio_31",
    "freq_model_max_delta_mags",
    "freq_model_min_delta_mags",
    "freq_model_phi1_phi2",
    "freq_n_alias",
    "freq_signif_ratio_21",
    "freq_signif_ratio_31",
    "freq_varrat",
    "freq_y_offset",
    "linear_trend",
    "medperc90_2p_p",
    "p2p_scatter_2praw",
    "p2p_scatter_over_mad",
    "p2p_scatter_pfold_over_mad",
    "p2p_ssqr_diff_over_var",
    "scatter_res_raw",
]

##########################################################

def rolling_window_celsium(series, window):
  data_out3 = []
  for i in range(len(series)-window):
      example = ((series[i:i+window]))
      fset_cesium = featurize.featurize_time_series(times=example["timestamp"],values=example["m3"].values, errors=None,features_to_use=features_to_use_all)
      new_elements_values_reshaped = np.squeeze(fset_cesium.values) 
      data_featuredf = pd.DataFrame(new_elements_values_reshaped)
      data_out3.append(data_featuredf.T.values)
  data_out4 = np.squeeze(data_out3) 
  df2 = pd.DataFrame((data_out4))
  return df2



#########################################################


def recursive_multistep_forecasting_TsCesium(DataCesium, model, horizon):
    seriec = DataCesium
    preds = []
    for i in range(horizon):
        example = seriec
        fset_cesium = featurize.featurize_time_series(times=example["timestamp"],values=example["y"].values, errors=None,features_to_use=features_to_use_all)
        new_elements_values_reshaped = np.squeeze(fset_cesium.values) 
        data_featuredf = pd.DataFrame(new_elements_values_reshaped)
        data_featuredf.fillna(0, inplace=True)
        data_featuredf.replace([float('inf'), -float('inf')], 0, inplace=True)
        pred = model.predict(data_featuredf.T)[0]
        preds.append(pred)
        series2 = seriec.drop(seriec.index[0])
        last_timestamp = series2['timestamp'].iloc[-1]
        year = int(str(last_timestamp)[:4])
        month = int(str(last_timestamp)[4:])
        if month == 12:
            next_year = year + 1
            next_month = 1
        else:
            next_year = year
            next_month = month + 1
        next_timestamp = next_year * 100 + next_month
        new_row = pd.DataFrame({'timestamp': [next_timestamp], 'y': [pred]})
        seriec = pd.concat([series2, new_row], ignore_index=True)
    return preds


#########################################################

In [182]:
############## TsCesium SAVE ##############

import pickle

horizon = 12
window = 12
features = 'TsCesium'

## Depende da janela e da Feature
if window == 12:
    FeatRegister = 398 
elif window ==24:
    FeatRegister = 386
else:
    FeatRegister = 374

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:

        # carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df

        outTsCelsium = rolling_window_celsium(series, window) 
        seriesy = series.iloc[12:]
        seriesy2= seriesy.reset_index()

        outTsCelsium = outTsCelsium.fillna(0)
        
        TimeStamp = df['timestamp'].tail(FeatRegister) 
        TimeStamp.reset_index(drop=True, inplace=True) 
 
        outTsCelsium.insert(0, 'timestamp', TimeStamp)

        folder_name = f'TsCesium'
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        outTsCelsium.to_csv(f'{folder_name}/FEAT_TsCesium_{product}_{window}_{estado}.csv', index=False)


KeyboardInterrupt: 

In [6]:
############## TsCesium TEST ##############

import pickle

horizon = 12
window = 12
features = 'TsCesium'

## Depende da janela e da Feature
if window == 12:
    FeatRegister = 398 
elif window ==24:
    FeatRegister = 386
else:
    FeatRegister = 374

#################################################
product = 'etanolhidratado'
estado = 'ac'
#################################################

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
# for product in products:
#     folder_path = f'./uf/{product}/'
#     # Read the CSV files and extract estado names
#     estados = read_csv_files(folder_path)
#     for estado in estados:

# carregamento do arquivo

df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
series = df

outTsCelsium = rolling_window_celsium(series, window) 

outTsCelsium = outTsCelsium.fillna(0)
outTsCelsium.replace([float('inf'), -float('inf')], 0, inplace=True)

outTsCelsium['y'] = df['m3'].tail(FeatRegister).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(outTsCelsium, horizon)

############## Regressores ##################

regr1 = LinearRegression()
regr2 = KNeighborsRegressor(n_neighbors = 3)
regr3 = XGBRegressor()
regr4 = SVR(kernel='rbf')
regr5 = RandomForestRegressor()
regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)

#### Preparar data Cesium ##########
DataCesium = df['timestamp'].head(FeatRegister) 
DataCesium = DataCesium.tail(horizon)
DataCesium.reset_index(drop=True, inplace=True)
DataCesium = pd.DataFrame(DataCesium)
tempy=y_train.tail(horizon)
tempy.reset_index(drop=True, inplace=True)
DataCesium['y']=tempy

#####################################

regr1.fit(X_train, y_train)
predictions1a = recursive_multistep_forecasting_TsCesium(DataCesium, regr1, horizon)

regr2.fit(X_train, y_train)
predictions2a = recursive_multistep_forecasting_TsCesium(DataCesium, regr2, horizon)

regr3.fit(X_train, y_train)
predictions3a = recursive_multistep_forecasting_TsCesium(DataCesium, regr3, horizon)

regr4.fit(X_train, y_train)
predictions4a = recursive_multistep_forecasting_TsCesium(DataCesium, regr4, horizon)

regr5.fit(X_train, y_train)
predictions5a = recursive_multistep_forecasting_TsCesium(DataCesium, regr5, horizon)

regr6.fit(X_train, y_train)
predictions6a = recursive_multistep_forecasting_TsCesium(DataCesium, regr6, horizon)

Valores_Reais = series['m3'].tail(horizon)
Valores_Reais = Valores_Reais.reset_index(drop=True)


mape_result1 = mape(Valores_Reais, predictions1a)
mape_result2 = mape(Valores_Reais, predictions2a)
mape_result3 = mape(Valores_Reais, predictions3a)
mape_result4 = mape(Valores_Reais, predictions4a)
mape_result5 = mape(Valores_Reais, predictions5a)
mape_result6 = mape(Valores_Reais, predictions6a)

pbe_result1 = pbe(Valores_Reais, predictions1a)
pbe_result2 = pbe(Valores_Reais, predictions2a)
pbe_result3 = pbe(Valores_Reais, predictions3a)
pbe_result4 = pbe(Valores_Reais, predictions4a)
pbe_result5 = pbe(Valores_Reais, predictions5a)
pbe_result6 = pbe(Valores_Reais, predictions6a)

pocid_result1 = pocid(Valores_Reais, predictions1a)
pocid_result2 = pocid(Valores_Reais, predictions2a)
pocid_result3 = pocid(Valores_Reais, predictions3a)
pocid_result4 = pocid(Valores_Reais, predictions4a)
pocid_result5 = pocid(Valores_Reais, predictions5a)
pocid_result6 = pocid(Valores_Reais, predictions6a)

predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
predictions6b = ', '.join(f"{item:f}" for item in predictions6a)
    
rows_data = [
    [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
    [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
    [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
    [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
    [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
    [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],          
]

# Salva Modelos
folder_path = f"./00-MODELS_UF_MENSAL-TsCesium/"
os.makedirs(folder_path, exist_ok=True)

with open(os.path.join(folder_path, f'TsCesium_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
    pickle.dump(regr2, fd1)
with open(os.path.join(folder_path, f'TsCesium_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
    pickle.dump(regr2, fd2)
with open(os.path.join(folder_path, f'TsCesium_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
    pickle.dump(regr3, fd3)
with open(os.path.join(folder_path, f'TsCesium_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
    pickle.dump(regr4, fd4)
with open(os.path.join(folder_path, f'TsCesium_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
    pickle.dump(regr5, fd5)
with open(os.path.join(folder_path, f'TsCesium_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
    pickle.dump(regr2, fd6)

# CSV Output VALORES REAIS
with open(f'TsCesium_{window}_output.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    for row_data in rows_data:
        writer.writerow(row_data)




In [178]:
############## TsCesium AUTO ##############

import pickle

horizon = 12
window = 12
features = 'TsCesium'

## Depende da janela e da Feature
if window == 12:
    FeatRegister = 398 
elif window ==24:
    FeatRegister = 386
else:
    FeatRegister = 374

#################################################
# product = 'etanolhidratado'
# estado = 'ac'
#################################################

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    for estado in estados:

# carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df

        outTsCelsium = rolling_window_celsium(series, window) 

        outTsCelsium = outTsCelsium.fillna(0)
        outTsCelsium.replace([float('inf'), -float('inf')], 0, inplace=True)

        outTsCelsium['y'] = df['m3'].tail(FeatRegister).reset_index(drop=True)

        X_train, X_test, y_train, y_test = train_test_split(outTsCelsium, horizon)

        ############## Regressores ##################

        regr1 = LinearRegression()
        regr2 = KNeighborsRegressor(n_neighbors = 3)
        regr3 = XGBRegressor()
        regr4 = SVR(kernel='rbf')
        regr5 = RandomForestRegressor()
        regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)

        #### Preparar data Cesium ##########
        DataCesium = df['timestamp'].head(FeatRegister) 
        DataCesium = DataCesium.tail(horizon)
        DataCesium.reset_index(drop=True, inplace=True)
        DataCesium = pd.DataFrame(DataCesium)
        tempy=y_train.tail(horizon)
        tempy.reset_index(drop=True, inplace=True)
        DataCesium['y']=tempy

        #####################################

        regr1.fit(X_train, y_train)
        predictions1a = recursive_multistep_forecasting_TsCesium(DataCesium, regr1, horizon)

        regr2.fit(X_train, y_train)
        predictions2a = recursive_multistep_forecasting_TsCesium(DataCesium, regr2, horizon)

        regr3.fit(X_train, y_train)
        predictions3a = recursive_multistep_forecasting_TsCesium(DataCesium, regr3, horizon)

        regr4.fit(X_train, y_train)
        predictions4a = recursive_multistep_forecasting_TsCesium(DataCesium, regr4, horizon)

        regr5.fit(X_train, y_train)
        predictions5a = recursive_multistep_forecasting_TsCesium(DataCesium, regr5, horizon)

        regr6.fit(X_train, y_train)
        predictions6a = recursive_multistep_forecasting_TsCesium(DataCesium, regr6, horizon)

        Valores_Reais = series['m3'].tail(horizon)
        Valores_Reais = Valores_Reais.reset_index(drop=True)


        mape_result1 = mape(Valores_Reais, predictions1a)
        mape_result2 = mape(Valores_Reais, predictions2a)
        mape_result3 = mape(Valores_Reais, predictions3a)
        mape_result4 = mape(Valores_Reais, predictions4a)
        mape_result5 = mape(Valores_Reais, predictions5a)
        mape_result6 = mape(Valores_Reais, predictions6a)

        pbe_result1 = pbe(Valores_Reais, predictions1a)
        pbe_result2 = pbe(Valores_Reais, predictions2a)
        pbe_result3 = pbe(Valores_Reais, predictions3a)
        pbe_result4 = pbe(Valores_Reais, predictions4a)
        pbe_result5 = pbe(Valores_Reais, predictions5a)
        pbe_result6 = pbe(Valores_Reais, predictions6a)

        pocid_result1 = pocid(Valores_Reais, predictions1a)
        pocid_result2 = pocid(Valores_Reais, predictions2a)
        pocid_result3 = pocid(Valores_Reais, predictions3a)
        pocid_result4 = pocid(Valores_Reais, predictions4a)
        pocid_result5 = pocid(Valores_Reais, predictions5a)
        pocid_result6 = pocid(Valores_Reais, predictions6a)

        predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
        predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
        predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
        predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
        predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
        predictions6b = ', '.join(f"{item:f}" for item in predictions6a)
            
        rows_data = [
            [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
            [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
            [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
            [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
            [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
            [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],          
        ]

        # Salva Modelos
        folder_path = f"./00-MODELS_UF_MENSAL-TsCesium/"
        os.makedirs(folder_path, exist_ok=True)

        with open(os.path.join(folder_path, f'TsCesium_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
            pickle.dump(regr2, fd1)
        with open(os.path.join(folder_path, f'TsCesium_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
            pickle.dump(regr2, fd2)
        with open(os.path.join(folder_path, f'TsCesium_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
            pickle.dump(regr3, fd3)
        with open(os.path.join(folder_path, f'TsCesium_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
            pickle.dump(regr4, fd4)
        with open(os.path.join(folder_path, f'TsCesium_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
            pickle.dump(regr5, fd5)
        with open(os.path.join(folder_path, f'TsCesium_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
            pickle.dump(regr2, fd6)

        # CSV Output VALORES REAIS
        with open(f'TsCesium_{window}_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for row_data in rows_data:
                writer.writerow(row_data)





KeyboardInterrupt: 